In [72]:
#Загружаем бибилиотекуи
#для работы с веб-запросами
import requests
#для парсинга страниц BeautifulSoup
from bs4 import BeautifulSoup
#для работы с датафреймами pandas
import pandas as pd
#для задержек
import time
#для чтоения файлов в формате json
import json

In [75]:
#ЗАДАНИЕ 1

#Ключевые слова для поиска
KEYWORDS = ['python', 'парсинг']

#Датафрейм для записи результатов
result = pd.DataFrame(columns = ['дата', 'заголовок', 'ссылка', 'текст_статьи'])

#Нужны куки, без них почему-то перестало работать!!!
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 OPR/77.0.4054.203',
    'Cookie': 'web_override=true; you_are_lucky_man=true; _ym_d=1625316267; _ym_uid=1586462892671260366; _ga=GA1.2.1786010787.1625316267; fl=ru; hl=ru; web_override_closed=true; tildauid=1625750714696.304602; _gid=GA1.2.1678597827.1626000237; _ym_isad=2; habr_web_home=ARTICLES_LIST_ALL; _gat=1; _gat_HGM=1',   
}

#Делаем GET-запрос на сайт
get_request = requests.get('https://habr.com/ru/all/', headers=headers)
#Разбиваем сплошной текст на отдельные составляющие
soup = BeautifulSoup(get_request.text, 'html.parser')
#Извлекаем превью статей (их всего 20 на странице)
articles_all = soup.find_all('div', class_='tm-article-snippet')


#Перебираем статьи по одной
for article in articles_all:
    #Выделяем конструкцию заголовка со ссылкой: внутри тега 'a' ищем class_='tm-article-snippet__title-link'
    title_link = article.find('a', class_='tm-article-snippet__title-link')
    #Выделяем текст заголовка внутри тега 'span'
    title = title_link.find('span').text
    #Выделяем ссылку как содержимое 'href' и дополняем адресом сайта 'https://habr.com'
    link = 'https://habr.com' + title_link.get('href')
    #Выделяем текст превью
    preview = article.find('div', class_ = 'tm-article-body tm-article-snippet__lead').text
    #Получаем статью
    article_soup = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
    #Получаем текст статьи
    article_text = article_soup.find('div', class_='tm-article-body').text
    #Получаем дату
    date = article_soup.find('span', class_='tm-article-snippet__datetime-published').find('time').get('title')
    #Задержка 
    time.sleep(0.2)    
    
    #Проверяем есть ли в заголовке, превью или текте статьи нужные нам слова
    #Для исключения путаницы с регистром переводим всё в нижний
    for word in KEYWORDS:
        if word.lower() in (title.lower() + preview.lower() + article_text.lower()):
            #Проверем нет ли уже строки с таким же заголовком
            if title not in result['заголовок'].values:
                #Создаёт строку и записываем её в результаты
                row = {'дата': date, 'заголовок': title, 'ссылка': link, 'текст_статьи': article_text}
                result = result.append(row, ignore_index=True)           

#Проверка результата
result

,дата,заголовок,ссылка,текст_статьи
0,"2021-07-13, 08:14",Плагин для ранжирования кода по важности или к...,https://habr.com/ru/company/hsespb/blog/567332/,Обычно в кодовой базе крупных проектов есть бо...
1,"2021-07-13, 00:37","Даты, время и часовые пояса: улучшения в .NET 6",https://habr.com/ru/company/otus/blog/567338/,Материал переведен. Ссылка на оригиналВ этой с...
2,"2021-07-12, 22:17",Всё о проекте «Спутниковый интернет Starlink»....,https://habr.com/ru/post/567258/,Предлагаю ознакомиться с ранее размещенными ма...
3,"2021-07-12, 20:53",Тюнер для автонастройки магнитной петлевой ант...,https://habr.com/ru/company/skillfactory/blog/...,Представленный в этой инструкции тюнер магнитн...
4,"2021-07-12, 18:00",DRS как средство оптимизации размещения виртуа...,https://habr.com/ru/company/mailru/blog/565528/,\nSource\nПланировщик распределенных ресурсов ...


In [76]:
#ЗАДАНИЕ 2

#Список email-ов для проверки
EMAIL = ['xxx@x.ru', 'yyy@y.com']

#Датафрейм для записи результатов
result = pd.DataFrame(columns = ['дата утечки', 'источник утечки', 'описание утечки'])

#Адрес для post-запроса
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

#Заголовки для post-запроса, без них не работает
headers = {
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}

#Отравляем post-запрос
post_req = requests.post(url, data = json.dumps({'emailAddresses': EMAIL}), headers=headers)
#Получаем ответ в json-формате, преобразуем его в словарь
all_info = json.loads(post_req.text)

#В словаре all_info нас интересует раздел breaches с подробным описание утечек
#Перебираем записи по номеру утечек breach_id
for breach_id in all_info['breaches']:
    #Получаем словрь с информацией для одной утечки
    breach_info = all_info['breaches'][breach_id]
    #Дата утечки (берём дату как первые 10 символов)
    date = breach_info['publishDate'][:10]
    #Источник утечки
    source = breach_info['site']
    #Описание утечки
    description = breach_info['description']
    
    #Создаём строку и записываем её в результаты
    row = {'дата утечки': date, 'источник утечки': source, 'описание утечки': description}
    result = result.append(row, ignore_index=True)  

#Проверка результата
result

,дата утечки,источник утечки,описание утечки
0,2016-10-21,linkedin.com,"In 2012, online professional networking platfo..."
1,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,2016-10-29,vk.com,Popular Russian social networking platform VKo...
3,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
4,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major...
5,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
6,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
7,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
8,2017-03-15,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
9,2017-03-01,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."


In [117]:
#ЗАДАНИЕ 2 (Дополнительная часть )

#Датафрейм для записи результатов
result = pd.DataFrame(columns = ['дата поста', 'текст поста'])

#Токен для достуа к сайту
TOKEN = 

#На этот адрес отправляем get-запрос
REQUEST = 'https://api.vk.com/method/wall.get'

#Собираем посты с группы Нетологии
GROUP = 'netology'
#Параметры для get-запроса
params = {
    'access_token': TOKEN,
    'v': 5.84,
    'domain': GROUP,
    'count': 50
}

#Отравляем get-запрос, ответ в формате json
all_info = requests.get(REQUEST, params).json()
#Получаем список постов
post_list = all_info['response']['items']

#еребираем посты по одному
for post in post_list:
    #Дата поста (unixtime переводим в привычный формат)
    date = time.strftime('%Y-%m-%d, %H:%M', time.localtime(post['date']))
    #Текст поста
    text = post['text']
    
    #Создаём строку и записываем её в результаты
    row = {'дата поста': date, 'текст поста': text}
    result = result.append(row, ignore_index=True) 

#Проверка результата
result

,дата поста,текст поста
0,"2021-07-07, 19:30",Срочное сообщение от коллеги в пятницу вечером...
1,"2021-07-12, 20:43","По данным Mediascope, в России активно пользую..."
2,"2021-07-12, 12:41",27 июля начнётся бесплатный онлайн-марафон «Ма...
3,"2021-07-11, 19:15",Хороший специалист собирает знания из разных п...
4,"2021-07-11, 14:01",Сейчас принято быть продуктивным: строить карь...
5,"2021-07-10, 13:17",В современном мире бизнесу трудно выжить без и...
6,"2021-07-09, 19:30","На улице слишком жарко, поэтому заниматься спо..."
7,"2021-07-09, 12:11","Проблемы в карьере — это нормально, не стоит н..."
8,"2021-07-08, 19:56","Выбрать красивый фон и шрифт — ещё не значит, ..."
9,"2021-07-08, 12:47",У нас на Ютуб-канале появился новый формат вид...
